In [ ]:
!pip install torchtext==0.15.1
!pip install torch==2.1.0
!pip install transformers==4.27.1
!pip install datasets==2.17.0

In [ ]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

qa_dataset = load_dataset('squad', split='train').shard(num_shards=40, index=0)
qa_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2190
})

In [ ]:
import re

def text_normalize(text):
    text = re.sub(r'[^\w\s]', ' ', text)

    return text

# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(data):
    for item in data:
        yield tokenizer('<cls> ' + text_normalize(item['context']) + ' <sep> ' + text_normalize(item['question']))

# Create vocabulary
vocab = build_vocab_from_iterator(
    yield_tokens(qa_dataset),
    specials=['<unk>', '<pad>', '<bos>', '<eos>', '<sep>', '<cls>']
)
vocab.set_default_index(vocab['<unk>'])
vocab.get_stoi()

{'하': 26911,
 '큰': 26910,
 '소': 26905,
 '사람': 26904,
 '나라이름': 26901,
 '魄': 26900,
 '魂': 26899,
 '金陵邑': 26897,
 '越城': 26892,
 '豊臣秀吉': 26891,
 '義皇帝': 26889,
 '禅': 26885,
 '现代汉语通用字表': 26884,
 '浙江': 26880,
 '法王': 26879,
 '汉语水平考试': 26877,
 '水': 26875,
 '校尉': 26874,
 '李閏': 26873,
 '方块字': 26870,
 '平安': 26868,
 '小': 26867,
 '唐入り': 26863,
 '北軍': 26861,
 '凹田': 26860,
 '冶城': 26859,
 '下': 26856,
 'トワイライトプリンセス': 26854,
 'ゼルダの伝説': 26851,
 'ἥλιος': 26850,
 'โรงเร': 26845,
 'สลาม': 26844,
 'ยนศาสนาอ': 26843,
 'ब': 26842,
 'نصراني': 26840,
 'نصارى': 26839,
 'العربية': 26835,
 'языка': 26831,
 'хийума': 26829,
 'стиль': 26827,
 'русского': 26826,
 'рмонтов': 26825,
 'пу': 26824,
 'никола': 26823,
 'насекомое': 26822,
 'н': 26821,
 'мок': 26820,
 'михаи': 26819,
 'кий': 26816,
 'замо': 26812,
 'дов': 26810,
 'голь': 26807,
 'высо': 26805,
 'χαλκός': 26800,
 'φαναῖος': 26798,
 'τεχνικά': 26796,
 'πικρό': 26795,
 'πάνορμος': 26794,
 'λύκη': 26793,
 'λύκειος': 26792,
 'λύκειο': 26791,
 'λόγος': 26790,
 'λυκ

In [ ]:
MAX_SEQ_LEN = 512
PAD_IDX = vocab['<pad>']

def pad_and_truncate(input_ids, max_seq_len):
    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
    elif len(input_ids) < max_seq_len:
        input_ids += [PAD_IDX] * (max_seq_len - len(input_ids))

    return input_ids

def vectorize(question, context, answer):
    input_text = '<cls> ' + text_normalize(question) + ' <sep> ' + text_normalize(context)
    input_ids = [vocab[token] for token in tokenizer(input_text)]
    input_ids = pad_and_truncate(input_ids, MAX_SEQ_LEN)

    answer_ids = [vocab[token] for token in tokenizer(text_normalize(answer))]
    try:
        start_positions = input_ids.index(answer_ids[0])
        end_positions = start_positions + len(answer_ids) - 1
    except:
        start_positions = 0
        end_positions = 0

    input_ids = torch.tensor(input_ids, dtype=torch.long)
    start_positions = torch.tensor(start_positions, dtype=torch.long)
    end_positions = torch.tensor(end_positions, dtype=torch.long)

    return input_ids, start_positions, end_positions

In [ ]:
class QADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question_text = item['question']
        context_text = item['context']
        answer_text = item['answers']['text'][0]

        input_ids, start_positions, end_positions = vectorize(question_text, context_text, answer_text)

        return input_ids, start_positions, end_positions

In [ ]:
def decode(input_ids):
    return ' '.join([vocab.lookup_token(token) for token in input_ids])

In [ ]:
for item in qa_dataset:
    question_text = item['question']
    context_text = item['context']
    answer_text = item['answers']['text'][0]

    input_ids, start_positions, end_positions = vectorize(question_text, context_text, answer_text)
    print(input_ids)
    text = decode(input_ids)
    answer_span = input_ids[start_positions:end_positions+1]

    print(text)
    print(decode(answer_span))

    break

tensor([    5,    10,  1256,    48,     6,  1420,   587,  8195,  1478,     9,
         4080,     9,  9276,   244,     4, 10672,     6,   188,    34,    11,
          767,  1326,  5138,     6,   371,   380,    17,  1391,  4218,    13,
           11,  1993,  3680,     7,     6,  1420,   587,  1186,     9,   850,
            7,     6,   371,   380,     8,  6072,    26,    13,    11,  1279,
         3680,     7,  1105,    18,  1434, 26013,    18,     6,  3309, 26127,
         1169,  2109, 22332,   614,    10,     6,   371,   380,    13,     6,
         4142,     7,     6,  2401,  2348,  1186,  1024,     6,  4142,    13,
            6, 12012,    11, 12672,   222,     7,  3966,     8, 23630,    26,
           13,    11, 13475,     7,     6, 12012,    27,  9276,   244,    80,
            6,  1420,   587, 23759,  1376,    10,   645, 16126, 24746,     9,
         4080,    27,     6,   169,     7,     6,   371,  1701,     8,     9,
           11,   584,   282,    19,  5955,   116,   118,  4429, 

In [ ]:
BATCH_SIZE = 256
train_dataset = QADataset(qa_dataset)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
import math
import torch.nn as nn
import torch.optim as optim

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim,
                                          num_heads=num_heads)
        self.ffn = nn.Linear(in_features=embed_dim,
                             out_features=ff_dim)
        self.layernorm_1 = nn.LayerNorm(normalized_shape=embed_dim)
        self.layernorm_2 = nn.LayerNorm(normalized_shape=embed_dim)

    def forward(self, query, key, value):
        attn_output, _ = self.attn(query, key, value)
        out_1 = self.layernorm_1(query + attn_output)
        ffn_output = self.ffn(out_1)
        x = self.layernorm_2(out_1 + ffn_output)

        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]

        return x

class QAModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_heads, ff_dim, max_len):
        super(QAModel, self).__init__()
        self.input_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_encoder = PositionalEncoding(embedding_dim)
        self.transformer = TransformerBlock(embedding_dim, n_heads, ff_dim)

        self.start_linear = nn.Linear(ff_dim, 1)
        self.end_linear = nn.Linear(ff_dim, 1)

    def forward(self, text):
        input_embedded = self.input_embedding(text)
        input_embedded = self.pos_encoder(input_embedded)
        transformer_out = self.transformer(input_embedded, input_embedded, input_embedded)
        start_logits = self.start_linear(transformer_out).squeeze(-1)
        end_logits = self.end_linear(transformer_out).squeeze(-1)

        return start_logits, end_logits

# Model parameters
EMBEDDING_DIM = 64
FF_DIM = 64
N_HEADS = 8
VOCAB_SIZE = len(vocab)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = QAModel(VOCAB_SIZE, EMBEDDING_DIM, N_HEADS, FF_DIM, MAX_SEQ_LEN).to(device)

input = torch.randint(0, 10, size=(1, 10)).to(device)
print(input.shape)
model.eval()
with torch.no_grad():
    start_logits, end_logits = model(input)

print(start_logits.shape)

torch.Size([1, 10])
torch.Size([1, 10])


In [ ]:
LR = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm
EPOCHS = 150

model.train()
for epoch in tqdm(range(EPOCHS)):
    train_losses = []
    for idx, (input_ids, start_positions, end_positions) in enumerate(train_loader):
        input_ids = input_ids.to(device)
        start_positions = start_positions.to(device)
        end_positions = end_positions.to(device)
        optimizer.zero_grad()
        start_logits, end_logits = model(input_ids)
        start_loss = criterion(start_logits, start_positions)
        end_loss = criterion(end_logits, end_positions)
        total_loss = (start_loss + end_loss) / 2
        total_loss.backward()
        optimizer.step()
        train_losses.append(total_loss.item())
    train_loss = sum(train_losses) / len(train_losses)
    print(f'EPOCH {epoch + 1}\tTraining Loss: {train_loss}')

  1%|          | 1/150 [00:02<05:39,  2.28s/it]

EPOCH 1	Training Loss: 5.084600501590305


  1%|▏         | 2/150 [00:04<05:01,  2.03s/it]

EPOCH 2	Training Loss: 4.619819164276123


  2%|▏         | 3/150 [00:05<04:45,  1.94s/it]

...

EPOCH 148	Training Loss: 0.6408179733488295


 99%|█████████▉| 149/150 [04:41<00:01,  1.93s/it]

EPOCH 149	Training Loss: 0.6240955922338698


100%|██████████| 150/150 [04:43<00:00,  1.89s/it]

EPOCH 150	Training Loss: 0.6417749457889133


In [ ]:
model.eval()
with torch.no_grad():
    sample = qa_dataset[150]
    context, question, answer = sample['context'], sample['question'], sample['answers']['text'][0]
    # context = 'Jane is a student and she is from Canada'
    # question = 'What does Jane do?'
    # answer = 'student'
    context = text_normalize(context)
    question = text_normalize(question)
    answer = text_normalize(answer)
    input_ids, start_positions, end_positions = vectorize(question, context, answer)
    input_ids = input_ids.to(device)
    start_positions = start_positions.to(device)
    end_positions = end_positions.to(device)
    input_ids = input_ids.unsqueeze(0)
    start_logits, end_logits = model(input_ids)

    offset = len(tokenizer(question)) + 2
    start_position = torch.argmax(start_logits, dim=1).cpu().numpy()[0]
    end_position = torch.argmax(end_logits, dim=1).cpu().numpy()[0]

    start_position -= offset
    end_position -= offset

    start_position = max(start_position, 0)
    end_position = min(end_position, len(tokenizer(context)) - 1)

    if end_position >= start_position:
        # Extract the predicted answer span
        context_tokens = tokenizer(context)
        predicted_answer_tokens = context_tokens[start_position:end_position + 1]
        predicted_answer = ' '.join(predicted_answer_tokens)
    else:
        predicted_answer = ''

    print(f'Context: {context}')
    print(f'Question: {question}')
    print(f'Start position: {start_position}')
    print(f'End position: {end_position}')
    print(f'Answer span: {predicted_answer}')
    print(answer)

Context: The College Dropout was eventually issued by Roc A Fella in February 2004  shooting to number two on the Billboard 200 as his debut single   Through the Wire  peaked at number fifteen on the Billboard Hot 100 chart for five weeks   Slow Jamz   his second single featuring Twista and Jamie Foxx  became an even bigger success  it became the three musicians  first number one hit  The College Dropout received near universal critical acclaim from contemporary music critics  was voted the top album of the year by two major music publications  and has consistently been ranked among the great hip hop works and debut albums by artists   Jesus Walks   the album s fourth single  perhaps exposed West to a wider audience  the song s subject matter concerns faith and Christianity  The song nevertheless reached the top 20 of the Billboard pop charts  despite industry executives  predictions that a song containing such blatant declarations of faith would never make it to radio  The College Dro